In [26]:
from bisect import bisect_left
from itertools import accumulate

import utils
inp = utils.get_input(2025,4,"input")
inp = [[c=="@" for c in r] for r in inp]

In [27]:
from copy import deepcopy
from math import prod
def findCells(map):
    m,n = len(inp),len(inp[0])
    def getCell(x,y):
        if 0 <= x < n and 0 <= y < m:
            return map[y][x]
        return False

    def getSurr(x,y):
        count = 0
        for nx in [-1,1,0]:
            for ny in [-1,1,0]:
                if nx == ny == 0:
                    continue
                count += getCell(x+nx,y+ny)
        return count
    coords = []
    for y in range(m):
        for x in range(n):
            if getSurr(x,y) < 4 and getCell(x,y):
                coords.append((x,y))
                map[y][x] = False
    return coords

map = deepcopy(inp)
p1,p2 = 0,0
for i in range(len(inp)*len(inp[0])):
    coords = findCells(map)
    coordCount = len(coords)
    if i == 0:
        p1 = coordCount
    p2 += coordCount
    if coordCount==0:
        break

print("p1",p1)
print("p2",p2)

p1 3097
p2 8910


In [30]:
from viz.VideoGenerator import VideoGenerator
from viz.vizUtils import lerp_gradient_eased
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)
ORANGE = (255, 165, 0)
RED = (255, 0, 0)
BLACK = (0, 0, 0)

BACKGROUND = BLACK
ITEMS = WHITE
BACKGROUND_CODE = 999
ITEM_CODE = 998
palette = {
    BACKGROUND_CODE: BACKGROUND,
    ITEM_CODE: ITEMS,
}

MILLIS = 50
SECOND = 1000//MILLIS

colours = [WHITE, YELLOW, ORANGE, RED,BLACK][::-1]
LIFETIME = (SECOND*1)//2 # 2 sec lifetime
lerp_colours = []
for x in range(LIFETIME+1):
    grad_colour = lerp_gradient_eased(x,LIFETIME,colours)
    palette[x] = grad_colour

map = deepcopy(inp)
m,n = len(inp),len(inp[0])
lifetime_map = [[LIFETIME+1 for x in range(n)] for y in range(m)]

vg = VideoGenerator(name="05_2025_v3",colour_scheme=palette)
p1,p2 = 0,0

def getFrame(map,lifetime_map):
    arr = []
    for y in range(m):
        row = []
        for x in range(n):
            cell = int(map[y][x])
            colour_code = 0
            if cell:
                colour_code=ITEM_CODE
            else:
                lifetime = lifetime_map[y][x]
                if lifetime <= LIFETIME:
                    colour_code= lifetime
                else:
                    colour_code= BACKGROUND_CODE
            row.append(colour_code)
        arr.append(row)
    return arr

def updateLifetimes(lifetime_map):
    for y in range(m):
        for x in range(n):
            if 0<lifetime_map[y][x] <= LIFETIME:
                lifetime_map[y][x] -= 1





startingFrame = getFrame(map,lifetime_map)
for _ in range(SECOND):
    vg.add_frame(startingFrame)

for i in range(len(inp)*len(inp[0])):
    coords = findCells(map)
    coordCount = len(coords)
    if i == 0:
        p1 = coordCount
    p2 += coordCount
    if coordCount==0:
        break

    for x,y in coords:
        lifetime_map[y][x] = LIFETIME

    frame = getFrame(map,lifetime_map)

    vg.add_frame(frame)
    updateLifetimes(lifetime_map)


for _ in range(LIFETIME*2):
    endingFrame = getFrame(map,lifetime_map)
    vg.add_frame(endingFrame)
    updateLifetimes(lifetime_map)
vg.render_gif(MILLIS,scale=4)
